In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/ETMC/")
os.getcwd()

Mounted at /content/gdrive


'/content/gdrive/My Drive/ETMC'

In [ ]:
!pip install vision-mamba

ERROR: Operation cancelled by user


#数据库部分

1.   初始化项
2.   安装



In [ ]:
import torch.utils.data

class DataProvider():

    def __init__(self, cfg, dataset, batch_size=None, shuffle=True):
        super().__init__()
        self.dataset = dataset
        if batch_size is None:
            batch_size = cfg.BATCH_SIZE
        self.dataloader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=int(cfg.WORKERS),
            drop_last=False)

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for i, data in enumerate(self.dataloader):
            yield data

In [ ]:
import os.path
import random

import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
from torchvision.datasets.folder import make_dataset
import sys

ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch

from torchvision.transforms import functional as F
import copy


class AlignedConcDataset:

    def __init__(self, cfg, data_dir=None, transform=None, labeled=True):
        self.cfg = cfg
        self.transform = transform
        self.data_dir = data_dir
        self.labeled = labeled

        self.classes, self.class_to_idx = find_classes(self.data_dir)
        #假设 self.classes 是一个包含类别名称的列表，如 ['cat', 'dog', 'bird']，那么运行这行代码后，self.int_to_class 将成为以下字典：{0: 'cat', 1: 'dog', 2: 'bird'}
        self.int_to_class = dict(zip(range(len(self.classes)), self.classes))
        #函数将返回一个包含文件路径和相应类别索引的元组的列表，并找到带有PNG后缀的文件。
        self.imgs = make_dataset(self.data_dir, self.class_to_idx, 'png')

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        if self.labeled:
            img_path, label = self.imgs[index]
        else:
            img_path = self.imgs[index]

        #img_path 是一个包含文件路径的字符串，如 /path/to/images/image.jpg，那么 os.path.basename(img_path) 将返回 'image.jpg'，这是文件路径中的文件名部分。
        img_name = os.path.basename(img_path)
        AB_conc = Image.open(img_path).convert('RGB')

        # split RGB and Depth as A and B
        w, h = AB_conc.size
        w2 = int(w / 2)
        #按照宽度进行切割
        if w2 > self.cfg.FINE_SIZE:
            #使用了Pillow（或Python Imaging Library，PIL）库中的 crop 方法，并提供了一个矩形区域的坐标范围作为参数。这个矩形由左上角点 (0, 0) 和右下角点 (w2, h) 定
            #Image.BICUBIC：这是一个插值方法的参数。在这里，它指定了使用双三次插值方法进
            A = AB_conc.crop((0, 0, w2, h)).resize((self.cfg.LOAD_SIZE, self.cfg.LOAD_SIZE), Image.BICUBIC)
            B = AB_conc.crop((w2, 0, w, h)).resize((self.cfg.LOAD_SIZE, self.cfg.LOAD_SIZE), Image.BICUBIC)
        else:
            A = AB_conc.crop((0, 0, w2, h))
            B = AB_conc.crop((w2, 0, w, h))

        if self.labeled:
            sample = {'A': A, 'B': B, 'img_name': img_name, 'label': label}
        else:
            sample = {'A': A, 'B': B, 'img_name': img_name}

        if self.transform:
            sample['A'] = self.transform(sample['A'])
            sample['B'] = self.transform(sample['B'])

        return sample


class RandomCrop(transforms.RandomCrop):

    def __call__(self, sample):
        A, B = sample['A'], sample['B']

        #F.pad是pytorch内置的tensor扩充函数，便于对数据集图像或中间层特征进行维度扩充
        if self.padding > 0:
            A = F.pad(A, self.padding)
            B = F.pad(B, self.padding)

        # pad the width if needed
        if self.pad_if_needed and A.size[0] < self.size[1]:
            A = F.pad(A, (int((1 + self.size[1] - A.size[0]) / 2), 0))
            B = F.pad(B, (int((1 + self.size[1] - B.size[0]) / 2), 0))
        # pad the height if needed
        if self.pad_if_needed and A.size[1] < self.size[0]:
            A = F.pad(A, (0, int((1 + self.size[0] - A.size[1]) / 2)))
            B = F.pad(B, (0, int((1 + self.size[0] - B.size[1]) / 2)))

        i, j, h, w = self.get_params(A, self.size)
        sample['A'] = F.crop(A, i, j, h, w)
        sample['B'] = F.crop(B, i, j, h, w)

        # _i, _j, _h, _w = self.get_params(A, self.size)
        # sample['A'] = F.crop(A, i, j, h, w)
        # sample['B'] = F.crop(B, _i, _j, _h, _w)

        return sample


class CenterCrop(transforms.CenterCrop):

    def __call__(self, sample):
        A, B = sample['A'], sample['B']
        sample['A'] = F.center_crop(A, self.size)
        sample['B'] = F.center_crop(B, self.size)
        return sample


class RandomHorizontalFlip(transforms.RandomHorizontalFlip):

    def __call__(self, sample):
        A, B = sample['A'], sample['B']
        if random.random() > 0.5:
            A = F.hflip(A)
            B = F.hflip(B)

        sample['A'] = A
        sample['B'] = B

        return sample


def find_classes(dir):
    """
    Finds the class folders in a dataset.

    Args:
        dir (string): Root directory path.

    Returns:
        tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.

    Ensures:
        No class is a subdirectory of another.
    """
    #在表达式 sys.version_info >= (3, 5) 中，它检查Python解释器的版本是否大于或等于 3.5。
    if sys.version_info >= (3, 5):
        # Faster and available in Python 3.5 and above
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
    else:
        classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


class Resize(transforms.Resize):

    def __call__(self, sample):
        A, B = sample['A'], sample['B']
        h = self.size[0]
        w = self.size[1]

        sample['A'] = F.resize(A, (h, w))
        sample['B'] = F.resize(B, (h, w))

        return sample


class ToTensor(object):
    def __call__(self, sample):
        A, B = sample['A'], sample['B']

        # if isinstance(sample, dict):
        #     for key, value in sample:
        #         _list = sample[key]
        #         sample[key] = [F.to_tensor(item) for item in _list]

        sample['A'] = F.to_tensor(A)
        sample['B'] = F.to_tensor(B)

        return sample


class Normalize(transforms.Normalize):

    def __call__(self, sample):
        A, B = sample['A'], sample['B']
        sample['A'] = F.normalize(A, self.mean, self.std)
        sample['B'] = F.normalize(B, self.mean, self.std)

        return sample


class Lambda(transforms.Lambda):

    def __call__(self, sample):
        return self.lambd(sample)

模型部分

1.   基础模型
2.   图像编码



#backbone是Res


In [ ]:
import torch
import torch.nn as nn
import torchvision

class ImageEncoder(nn.Module):
    def __init__(self, args):
        super(ImageEncoder, self).__init__()
        self.args = args

        model = torchvision.models.resnet18(pretrained=True)
        #遍历函数的子层，这里即是features和classifier，然后把除去最后一层的层放到一个列表
        modules = list(model.children())[:-1]
        #变量前加一个星号*，目的是将该list变量拆解开多个独立的参数，传入函数中
        self.model = nn.Sequential(*modules)
        pool_func = (
            nn.AdaptiveAvgPool2d
            if args.img_embed_pool_type == "avg"
            else nn.AdaptiveMaxPool2d
        )

        if args.num_image_embeds in [1, 2, 3, 5, 7]:
            self.pool = pool_func((args.num_image_embeds, 1))
        elif args.num_image_embeds == 4:
            self.pool = pool_func((2, 2))
        elif args.num_image_embeds == 6:
            self.pool = pool_func((3, 2))
        elif args.num_image_embeds == 8:
            self.pool = pool_func((4, 2))
        elif args.num_image_embeds == 9:
            self.pool = pool_func((3, 3))

    def forward(self, x):
        # Bx3x224x224 -> Bx2048x7x7 -> Bx2048xN -> BxNx2048
       out = self.model(x)
       #print('model out',out.shape)
       out = self.pool(out)
       #print('pool out',out.shape)
       out = torch.flatten(out, start_dim=2)
       #print('flatten out',out.shape)
       out = out.transpose(1, 2).contiguous()
       #print('transpose out',out.shape)
       return out  # BxNx2048



#backbone更换为Mamba

In [ ]:
import torch
import torch.nn as nn
import torchvision
from vision_mamba import Vim

class ImageEncoder(nn.Module):
    def __init__(self, args):
        super(ImageEncoder, self).__init__()
        self.args = args
        #加载VisionMamba,这里输出类改为了和Res一致的args.img_hidden_sz，后续需要根据训练结果再看看
        model = Vim(
            dim=256,  # Dimension of the transformer model
            dt_rank=32,  # Rank of the dynamic routing matrix
            dim_inner=256,  # Inner dimension of the transformer model
            d_state=256,  # Dimension of the state vector
            num_classes=args.img_hidden_sz,  # Number of output classes
            image_size=args.FINE_SIZE,  # Size of the input image
            patch_size=16,  # Size of each image patch
            channels=3,  # Number of input channels
            dropout=0.1,  # Dropout rate
            depth=12,  # Depth of the transformer model
        )
        #modules = list(model.children())
        #变量前加一个星号*，目的是将该list变量拆解开多个独立的参数，传入函数中
        self.model = model
        #nn.AdaptiveAvgPool2d 对由多个输入平面组成的输入信号应用二维自适应平均池化。接受两个参数，分别为输出特征图的长和宽
        pool_func = (
            nn.AdaptiveAvgPool2d
            if args.img_embed_pool_type == "avg"
            else nn.AdaptiveMaxPool2d
        )

        if args.num_image_embeds in [1, 2, 3, 5, 7]:
            self.pool = pool_func((args.num_image_embeds, 1))
        elif args.num_image_embeds == 4:
            self.pool = pool_func((2, 2))
        elif args.num_image_embeds == 6:
            self.pool = pool_func((3, 2))
        elif args.num_image_embeds == 8:
            self.pool = pool_func((4, 2))
        elif args.num_image_embeds == 9:
            self.pool = pool_func((3, 3))

    def forward(self, x):
        # Bx3x224x224 -> Bx2048x7x7 -> Bx2048xN -> BxNx2048
        out = self.model(x)
        out = self.pool(out)
        #torch.flatten 作用是改变张量的维度和维数，从指定的维度开始将后面维度的维数全部展成一个维度，新的维数就是被展开的所有维度的维数的乘积。是按照x的第2个维度拼接
        out = torch.flatten(out, start_dim=2)
        #transpose交换out的第一个维度和第二个维度，
        #调用contiguous()时，会强制拷贝一份tensor，让它的布局和从头创建的一模一样，但是两个tensor完全没有联系，
        #更改其中一个的值不会影像到里一个。
        out = out.transpose(1, 2).contiguous()
        return out  # BxNx2048

#一般狄利克雷函数

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# loss function
def HD(alpha, c):
    a = 2.0
    b = a/(a-1)
    beta = torch.ones((1, c)).cuda()

    F1_1 = torch.sum(torch.lgamma(a * alpha + beta), dim=1, keepdim=True) - torch.lgamma(torch.sum((a * alpha + beta), dim=1, keepdim=True))
    F2_1 = torch.sum(torch.lgamma((b + 1) * beta), dim=1, keepdim=True) - torch.lgamma(torch.sum(((b + 1) * beta), dim=1, keepdim=True))
    F3_1 = torch.sum(torch.lgamma(alpha + 2 * beta), dim=1, keepdim=True) - torch.lgamma(torch.sum((alpha + 2 * beta), dim=1, keepdim=True))

    hd1 = 1/a * F1_1 + 1/b * F2_1 - F3_1
    print('f1:{},f2:{},f3:{},hd1:{}'.format(F1_1,F2_1,F3_1,hd1))
    a = b
    b = a/(a-1)
    F1_2 = torch.sum(torch.lgamma(a * alpha + beta), dim=1, keepdim=True) - torch.lgamma(torch.sum((a * alpha + beta), dim=1, keepdim=True))
    F2_2 = torch.sum(torch.lgamma((b + 1) * beta), dim=1, keepdim=True) - torch.lgamma(torch.sum(((b + 1) *  beta), dim=1, keepdim=True))
    F3_2 = torch.sum(torch.lgamma(alpha + 2 * beta), dim=1, keepdim=True) - torch.lgamma(torch.sum((alpha + 2 * beta), dim=1, keepdim=True))

    hd2 = 1/a * F1_2 + 1/b * F2_2 - F3_2

    hd = 1/2 * (hd1+hd2)
    return hd

def ce_loss(p, alpha, c, global_step, annealing_step):
    #print(alpha)
    S = torch.sum(alpha, dim=1, keepdim=True)
    E = alpha - 1
    #将标签转成one-hot编码
    label = F.one_hot(p, num_classes=c)
    #print("Label shape:", label.shape)
    #print("S shape:", S.shape)
    #print("Alpha shape:", alpha.shape)
    A = torch.sum(label * (torch.digamma(S) - torch.digamma(alpha)), dim=1, keepdim=True)

    annealing_coef = min(1, global_step / annealing_step)
    alp = E * (1 - label) + 1
    B = annealing_coef * HD(alp, c)
    #print('A:{},HD:{}'.format(A,B))
    print('A+B:{}'.format(torch.mean((A + B))))
    return torch.mean((A + B))


class TMC(nn.Module):
    def __init__(self, args):
        super(TMC, self).__init__()
        self.args = args
        #获得模型初始化参数，其中池化层是自定义的
        self.rgbenc = ImageEncoder(args)
        self.depthenc = ImageEncoder(args)
        depth_last_size = args.img_hidden_sz * args.num_image_embeds
        rgb_last_size = args.img_hidden_sz * args.num_image_embeds
        #nn.ModuleList，它是一个储存不同 module，并自动将每个 module 的 parameters 添加到网络之中的容器。
        self.clf_depth = nn.ModuleList()
        self.clf_rgb = nn.ModuleList()
        for hidden in args.hidden:
            self.clf_depth.append(nn.Linear(depth_last_size, hidden))
            self.clf_depth.append(nn.ReLU())
            self.clf_depth.append(nn.Dropout(args.dropout))
            depth_last_size = hidden
        self.clf_depth.append(nn.Linear(depth_last_size, args.n_classes))

        for hidden in args.hidden:
            self.clf_rgb.append(nn.Linear(rgb_last_size, hidden))
            self.clf_rgb.append(nn.ReLU())
            self.clf_rgb.append(nn.Dropout(args.dropout))
            rgb_last_size = hidden
        self.clf_rgb.append(nn.Linear(rgb_last_size, args.n_classes))

    def DS_Combin_two(self, alpha1, alpha2):
        # Calculate the merger of two DS evidences
        alpha = dict()
        alpha[0], alpha[1] = alpha1, alpha2
        b, S, E, u = dict(), dict(), dict(), dict()
        for v in range(2):
            #计算所有参数的和然后保留这个维度
            S[v] = torch.sum(alpha[v], dim=1, keepdim=True)
            E[v] = alpha[v] - 1
            #将单个维度扩大成更大维度，返回一个新的tensor
            b[v] = E[v] / (S[v].expand(E[v].shape))
            u[v] = self.args.n_classes / S[v]

        # b^0 @ b^(0+1)   torch.bmm计算两个tensor的矩阵乘法，torch.bmm(a,b),tensor a 的size为(b,h,w),tensor b的size为(b,w,m)
        #也就是说两个tensor的第一维是相等的，然后第一个数组的第三维和第二个数组的第二维度要求一样
        bb = torch.bmm(b[0].view(-1, self.args.n_classes, 1), b[1].view(-1, 1, self.args.n_classes))
        # b^0 * u^1
        #torch.mul是矩阵a和b对应位相乘，a和b的维度必须相等，比如a的维度是(1, 2)，b的维度是(1, 2)，返回的仍是(1, 2)的矩阵
        uv1_expand = u[1].expand(b[0].shape)
        bu = torch.mul(b[0], uv1_expand)
        # b^1 * u^0
        uv_expand = u[0].expand(b[0].shape)
        ub = torch.mul(b[1], uv_expand)
        # calculate K
        bb_sum = torch.sum(bb, dim=(1, 2), out=None)  #计算bb内所有参数的和
        bb_diag = torch.diagonal(bb, dim1=-2, dim2=-1).sum(-1)  #求bb内对角线附件的元素，并将最后一个维度求和
        # bb_diag1 = torch.diag(torch.mm(b[v], torch.transpose(b[v+1], 0, 1)))
        K = bb_sum - bb_diag

        # calculate b^a
        b_a = (torch.mul(b[0], b[1]) + bu + ub) / ((1 - K).view(-1, 1).expand(b[0].shape))
        # calculate u^a
        u_a = torch.mul(u[0], u[1]) / ((1 - K).view(-1, 1).expand(u[0].shape))
        # test = torch.sum(b_a, dim = 1, keepdim = True) + u_a #Verify programming errors

        # calculate new S
        S_a = self.args.n_classes / u_a
        # calculate new e_k
        e_a = torch.mul(b_a, S_a.expand(b_a.shape))
        alpha_a = e_a + 1
        return alpha_a

    def forward(self, rgb, depth):
        #首先输入depth数据到resnet18分类网络中，然后将分类结果作为预训练的结果扁平化处理
        depth = self.depthenc(depth)
        depth = torch.flatten(depth, start_dim=1)
        #首先输入rgb数据到resnet18分类网络中，然后将分类结果作为预训练的结果扁平化处理
        rgb = self.rgbenc(rgb)
        rgb = torch.flatten(rgb, start_dim=1)

        #对通过模型取得分类结果
        depth_out = depth
        for layer in self.clf_depth:
            depth_out = layer(depth_out)
        rgb_out = rgb
        for layer in self.clf_rgb:
            rgb_out = layer(rgb_out)

        #Softplus函数可以看作是ReLU激活函数的平滑代替。
        depth_evidence, rgb_evidence = F.softplus(depth_out), F.softplus(rgb_out)
        depth_alpha, rgb_alpha = depth_evidence+1, rgb_evidence+1
        depth_rgb_alpha = self.DS_Combin_two(depth_alpha, rgb_alpha)
        return depth_alpha, rgb_alpha, depth_rgb_alpha


class ETMC(TMC):
    def __init__(self, args):
        super(ETMC, self).__init__(args)
        #这一部分需要研究一下
        last_size = args.img_hidden_sz * args.num_image_embeds + args.img_hidden_sz * args.num_image_embeds
        self.clf = nn.ModuleList()
        for hidden in args.hidden:
            self.clf.append(nn.Linear(last_size, hidden))
            self.clf.append(nn.ReLU())
            self.clf.append(nn.Dropout(args.dropout))
            last_size = hidden
        self.clf.append(nn.Linear(last_size, args.n_classes))

    def forward(self, rgb, depth):
       # print("depth shape:", depth.shape)
        depth = self.depthenc(depth)
       # print("depthenc shape:", depth.shape)
        depth = torch.flatten(depth, start_dim=1)
        #print("flatted depth shape:", depth.shape)
        rgb = self.rgbenc(rgb)
        rgb = torch.flatten(rgb, start_dim=1)
        depth_out = depth
        for layer in self.clf_depth:
            depth_out = layer(depth_out)
        #print("depth_out shape:", depth_out.shape)
        rgb_out = rgb
        for layer in self.clf_rgb:
            rgb_out = layer(rgb_out)

        #按列拼接数据
        pseudo_out = torch.cat([rgb, depth], -1)
        for layer in self.clf:
            pseudo_out = layer(pseudo_out)

        depth_evidence, rgb_evidence, pseudo_evidence = F.softplus(depth_out), F.softplus(rgb_out), F.softplus(pseudo_out)
        #print("depth_evidence shape:", depth_evidence.shape)
        depth_alpha, rgb_alpha, pseudo_alpha = depth_evidence+1, rgb_evidence+1, pseudo_evidence+1
        depth_rgb_alpha = self.DS_Combin_two(self.DS_Combin_two(depth_alpha, rgb_alpha), pseudo_alpha)
        return depth_alpha, rgb_alpha, pseudo_alpha, depth_rgb_alpha


#光滑狄利克雷函数

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# loss function
def HD(alpha, c):
    a = 2.0
    b = a/(a-1)
    beta = torch.ones((1, c)).cuda()

    F1_1 = torch.lgamma(torch.sum(a * (alpha - 1) + 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(a * (alpha - 1) + 1), dim=1, keepdim=True)
    F2_1 = torch.lgamma(torch.sum(b * (beta - 1) + 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(b * (beta - 1) + 1), dim=1, keepdim=True)
    F3_1 = torch.lgamma(torch.sum(alpha + beta - 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(alpha + beta - 1), dim=1, keepdim=True)
    # Combine the terms into the final result
    hd1 = (1 / a) * F1_1 + (1 / b) * F2_1 - F3_1
    #print('f1:{},f2:{},f3:{},hd1:{}'.format(F1_1,F2_1,F3_1,hd1))
    a = b
    b = a/(a-1)
    F1_2 = torch.lgamma(torch.sum(a * (alpha - 1) + 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(a * (alpha - 1) + 1), dim=1, keepdim=True)
    F2_2 = torch.lgamma(torch.sum(b * (beta - 1) + 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(b * (beta - 1) + 1), dim=1, keepdim=True)
    F3_2 = torch.lgamma(torch.sum(alpha + beta - 1, dim=1, keepdim=True)) - torch.sum(torch.lgamma(alpha + beta - 1), dim=1, keepdim=True)

    hd2 = (1 / a) * F1_2 + (1 / b) * F2_2 - F3_2

    hd = -1/2 * (hd1+hd2)
    #hd = hd1
    return hd

def ce_loss(p, alpha, c, global_step, annealing_step):
    #print(alpha)
    S = torch.sum(alpha, dim=1, keepdim=True)
    E = alpha - 1
    #将标签转成one-hot编码
    label = F.one_hot(p, num_classes=c)
    #print("Label shape:", label.shape)
    #print("S shape:", S.shape)
    #print("Alpha shape:", alpha.shape)
    A = torch.sum(label * (torch.digamma(S) - torch.digamma(alpha)), dim=1, keepdim=True)

    annealing_coef = min(1, global_step / annealing_step)
    alp = E * (1 - label) + 1
    B = annealing_coef * HD(alp, c)
    return torch.mean((A + B))


class TMC(nn.Module):
    def __init__(self, args):
        super(TMC, self).__init__()
        self.args = args
        #获得模型初始化参数，其中池化层是自定义的
        self.rgbenc = ImageEncoder(args)
        self.depthenc = ImageEncoder(args)
        depth_last_size = args.img_hidden_sz * args.num_image_embeds
        rgb_last_size = args.img_hidden_sz * args.num_image_embeds
        #nn.ModuleList，它是一个储存不同 module，并自动将每个 module 的 parameters 添加到网络之中的容器。
        self.clf_depth = nn.ModuleList()
        self.clf_rgb = nn.ModuleList()
        for hidden in args.hidden:
            self.clf_depth.append(nn.Linear(depth_last_size, hidden))
            self.clf_depth.append(nn.ReLU())
            self.clf_depth.append(nn.Dropout(args.dropout))
            depth_last_size = hidden
        self.clf_depth.append(nn.Linear(depth_last_size, args.n_classes))

        for hidden in args.hidden:
            self.clf_rgb.append(nn.Linear(rgb_last_size, hidden))
            self.clf_rgb.append(nn.ReLU())
            self.clf_rgb.append(nn.Dropout(args.dropout))
            rgb_last_size = hidden
        self.clf_rgb.append(nn.Linear(rgb_last_size, args.n_classes))

    def DS_Combin_two(self, alpha1, alpha2):
        # Calculate the merger of two DS evidences
        alpha = dict()
        alpha[0], alpha[1] = alpha1, alpha2
        b, S, E, u = dict(), dict(), dict(), dict()
        for v in range(2):
            #计算所有参数的和然后保留这个维度
            S[v] = torch.sum(alpha[v], dim=1, keepdim=True)
            E[v] = alpha[v] - 1
            #将单个维度扩大成更大维度，返回一个新的tensor
            b[v] = E[v] / (S[v].expand(E[v].shape))
            u[v] = self.args.n_classes / S[v]

        # b^0 @ b^(0+1)   torch.bmm计算两个tensor的矩阵乘法，torch.bmm(a,b),tensor a 的size为(b,h,w),tensor b的size为(b,w,m)
        #也就是说两个tensor的第一维是相等的，然后第一个数组的第三维和第二个数组的第二维度要求一样
        bb = torch.bmm(b[0].view(-1, self.args.n_classes, 1), b[1].view(-1, 1, self.args.n_classes))
        # b^0 * u^1
        #torch.mul是矩阵a和b对应位相乘，a和b的维度必须相等，比如a的维度是(1, 2)，b的维度是(1, 2)，返回的仍是(1, 2)的矩阵
        uv1_expand = u[1].expand(b[0].shape)
        bu = torch.mul(b[0], uv1_expand)
        # b^1 * u^0
        uv_expand = u[0].expand(b[0].shape)
        ub = torch.mul(b[1], uv_expand)
        # calculate K
        bb_sum = torch.sum(bb, dim=(1, 2), out=None)  #计算bb内所有参数的和
        bb_diag = torch.diagonal(bb, dim1=-2, dim2=-1).sum(-1)  #求bb内对角线附件的元素，并将最后一个维度求和
        # bb_diag1 = torch.diag(torch.mm(b[v], torch.transpose(b[v+1], 0, 1)))
        K = bb_sum - bb_diag

        # calculate b^a
        b_a = (torch.mul(b[0], b[1]) + bu + ub) / ((1 - K).view(-1, 1).expand(b[0].shape))
        # calculate u^a
        u_a = torch.mul(u[0], u[1]) / ((1 - K).view(-1, 1).expand(u[0].shape))
        # test = torch.sum(b_a, dim = 1, keepdim = True) + u_a #Verify programming errors

        # calculate new S
        S_a = self.args.n_classes / u_a
        # calculate new e_k
        e_a = torch.mul(b_a, S_a.expand(b_a.shape))
        alpha_a = e_a + 1
        return alpha_a

    def forward(self, rgb, depth):
        #首先输入depth数据到resnet18分类网络中，然后将分类结果作为预训练的结果扁平化处理
        depth = self.depthenc(depth)
        depth = torch.flatten(depth, start_dim=1)
        #首先输入rgb数据到resnet18分类网络中，然后将分类结果作为预训练的结果扁平化处理
        rgb = self.rgbenc(rgb)
        rgb = torch.flatten(rgb, start_dim=1)

        #对通过模型取得分类结果
        depth_out = depth
        for layer in self.clf_depth:
            depth_out = layer(depth_out)
        rgb_out = rgb
        for layer in self.clf_rgb:
            rgb_out = layer(rgb_out)

        #Softplus函数可以看作是ReLU激活函数的平滑代替。
        depth_evidence, rgb_evidence = F.softplus(depth_out), F.softplus(rgb_out)
        depth_alpha, rgb_alpha = depth_evidence+1, rgb_evidence+1
        depth_rgb_alpha = self.DS_Combin_two(depth_alpha, rgb_alpha)
        return depth_alpha, rgb_alpha, depth_rgb_alpha


class ETMC(TMC):
    def __init__(self, args):
        super(ETMC, self).__init__(args)
        #这一部分需要研究一下
        last_size = args.img_hidden_sz * args.num_image_embeds + args.img_hidden_sz * args.num_image_embeds
        self.clf = nn.ModuleList()
        for hidden in args.hidden:
            self.clf.append(nn.Linear(last_size, hidden))
            self.clf.append(nn.ReLU())
            self.clf.append(nn.Dropout(args.dropout))
            last_size = hidden
        self.clf.append(nn.Linear(last_size, args.n_classes))

    def forward(self, rgb, depth):
       # print("depth shape:", depth.shape)
        depth = self.depthenc(depth)
       # print("depthenc shape:", depth.shape)
        depth = torch.flatten(depth, start_dim=1)
        #print("flatted depth shape:", depth.shape)
        rgb = self.rgbenc(rgb)
        rgb = torch.flatten(rgb, start_dim=1)
        depth_out = depth
        for layer in self.clf_depth:
            depth_out = layer(depth_out)
        #print("depth_out shape:", depth_out.shape)
        rgb_out = rgb
        for layer in self.clf_rgb:
            rgb_out = layer(rgb_out)

        #按列拼接数据
        pseudo_out = torch.cat([rgb, depth], -1)
        for layer in self.clf:
            pseudo_out = layer(pseudo_out)

        depth_evidence, rgb_evidence, pseudo_evidence = F.softplus(depth_out), F.softplus(rgb_out), F.softplus(pseudo_out)
        #print("depth_evidence shape:", depth_evidence.shape)
        depth_alpha, rgb_alpha, pseudo_alpha = depth_evidence+1, rgb_evidence+1, pseudo_evidence+1
        depth_rgb_alpha = self.DS_Combin_two(self.DS_Combin_two(depth_alpha, rgb_alpha), pseudo_alpha)
        return depth_alpha, rgb_alpha, pseudo_alpha, depth_rgb_alpha


#工具函数 utils

In [ ]:
import logging
import time
from datetime import timedelta


class LogFormatter:
    def __init__(self):
        self.start_time = time.time()
    #round()用于数字的四舍五入。
    def format(self, record):
        #获取程序运行的时间
        elapsed_seconds = round(record.created - self.start_time)
        #levelname代表日志的输出级别，包括 ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')
        #time.strftime("%x %X")按年月日时间的方式输出
        #timedelta() 表示两个 date 对象或者 time 对象,或者 datetime 对象之间的时间间隔，精确到微秒。
        prefix = "%s - %s - %s" % (
            record.levelname,
            time.strftime("%x %X"),
            timedelta(seconds=elapsed_seconds),
        )
        #GetMessage的主要功能是从消息队列中“取出”消息，消息被取出以后，就从消息队列中将其删除；
        message = record.getMessage()
        message = message.replace("\n", "\n" + " " * (len(prefix) + 3))
        return "%s - %s" % (prefix, message)


def create_logger(filepath, args):
    # create log formatter,创建输出日志的格式
    log_formatter = LogFormatter()

    # create file handler and set level to debug
    file_handler = logging.FileHandler(filepath, "a")
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(log_formatter)

    # create console handler and set level to info
    #Handler对象负责将适当的日志消息（基于日志消息的严重性）分派到处理程序的指定目标。
    #ch=logging.StreamHandler() # 将日志写入控制台
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(log_formatter)

    # create logger and set level to debug
    logger = logging.getLogger()
    logger.handlers = []
    logger.setLevel(logging.DEBUG)
    #确保当前记录器进行记录时则不会调用上层记录器
    logger.propagate = False
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # reset logger elapsed time
    def reset_time():
        log_formatter.start_time = time.time()

    logger.reset_time = reset_time
    #利用sorted函数对可迭代对象进行排序，按照元素items的标签进行升序
    logger.info(
        "\n".join(
            "%s: %s" % (k, str(v))
            for k, v in sorted(dict(vars(args)).items(), key=lambda x: x[0])
        )
    )

    return logger

In [ ]:
import contextlib
import numpy as np
import random
import shutil
import os

import torch

#默认seed=1
def set_seed(seed):
    random.seed(seed)
    #生成随机数组
    np.random.seed(seed)
    #设置CPU生成随机数的种子，方便下次复现实验结果
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    ##为当前GPU设置随机种子；
    torch.cuda.manual_seed_all(seed)
    #每次返回的卷积算法将是确定的
    torch.backends.cudnn.deterministic = True
    #设置为True，会使得cuDNN来衡量自己库里面的多个卷积算法的速度，然后选择其中最快的那个卷积算法
    torch.backends.cudnn.benchmark = False


def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pt"):
    filename = os.path.join(checkpoint_path, filename)
    torch.save(state, filename)
    #shutil.copyfile(filename, dst):将名为filename的文件的内容（无元数据）复制到名为dst的文件中.dst必须是完整的目标文件名
    if is_best:
        shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))


def load_checkpoint(model, path):
    best_checkpoint = torch.load(path)
    #给模型对象加载训练好的模型参数，即加载模型参数
    model.load_state_dict(best_checkpoint["state_dict"])

#输出日志信息，包含训练一次对应的损失函数值，深度视角准确度，rgb色彩准确度
#logger.info 是消息级别，当输出这个级别的消息时，会输出后面规定格式的文件
def log_metrics(set_name, metrics, logger):
    logger.info(
        "{}: Loss: {:.5f} | depth_acc: {:.5f}, rgb_acc: {:.5f}".format(
            set_name, metrics["loss"], metrics["depth_acc"], metrics["rgb_acc"]
        )
    )


@contextlib.contextmanager
def numpy_seed(seed, *addl_seeds):
    """Context manager which seeds the NumPy PRNG with the specified seed and
    restores the state afterward"""
    #人为设置一个断点，可以将yield看成一个断点，程序在断点处结束运行，又从断点处开始运行
    if seed is None:
        yield
        return
    #返回数据的哈希值，并除以10的六次方
    if len(addl_seeds) > 0:
        seed = int(hash((seed, *addl_seeds)) % 1e6)
    #获取随机生成器 np.random的状态 ，一次来确保哈希生成器每次工作的值的是一样的
    state = np.random.get_state()
    np.random.seed(seed)
    try:
        yield
    finally:
        np.random.set_state(state)

#分别用ETMC训练 nyud2,先设置参数

In [ ]:
import argparse
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torchvision.transforms as transforms
import os
import torch
from torch.utils.data import DataLoader

def get_args(parser):
    parser.add_argument("--batch_sz", type=int, default=32)
    parser.add_argument("--data_path", type=str, default="/content/gdrive/My Drive/ETMC/datasets/nyud2/")
    parser.add_argument("--LOAD_SIZE", type=int, default=256)
    parser.add_argument("--FINE_SIZE", type=int, default=224)
    parser.add_argument("--dropout", type=float, default=0.1)
    parser.add_argument("--gradient_accumulation_steps", type=int, default=3)
    parser.add_argument("--hidden", nargs="*", type=int, default=[])
    parser.add_argument("--hidden_sz", type=int, default=768)
    parser.add_argument("--img_embed_pool_type", type=str, default="avg", choices=["max", "avg"])
    parser.add_argument("--img_hidden_sz", type=int, default=512)
    parser.add_argument("--include_bn", type=int, default=True)
    parser.add_argument("--lr", type=float, default=5e-4)
    parser.add_argument("--lr_factor", type=float, default=0.2)
    parser.add_argument("--lr_patience", type=int, default=30)
    parser.add_argument("--max_epochs", type=int, default=500)
    parser.add_argument("--n_workers", type=int, default=12)
    parser.add_argument("--name", type=str, default="resReleasedVersion")
    parser.add_argument("--num_image_embeds", type=int, default=1)
    parser.add_argument("--patience", type=int, default=30)
    parser.add_argument("--savedir", type=str, default="/content/gdrive/My Drive/ETMC/results/nyud2/")
    parser.add_argument("--seed", type=int, default=1)
    parser.add_argument("--n_classes", type=int, default=10)
    parser.add_argument("--annealing_epoch", type=int, default=10)
    parser.add_argument("--Holder_parge", type=float, default=1.5)
    parser.add_argument("--backbone", type=str, default='Res')
def get_optimizer(model, args):
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    return optimizer


def get_scheduler(optimizer, args):
    return optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "max", patience=args.lr_patience, verbose=True, factor=args.lr_factor
    )


def model_forward(i_epoch, model, args, ce_loss, batch):
    rgb, depth, tgt = batch['A'], batch['B'], batch['label']


    rgb, depth, tgt = rgb.cuda(), depth.cuda(), tgt.cuda()
    depth_alpha, rgb_alpha, pseudo_alpha, depth_rgb_alpha = model(rgb, depth)

    loss = ce_loss(tgt, depth_alpha, args.n_classes, i_epoch, args.annealing_epoch) + \
           ce_loss(tgt, rgb_alpha, args.n_classes, i_epoch, args.annealing_epoch) + \
           ce_loss(tgt, pseudo_alpha, args.n_classes, i_epoch, args.annealing_epoch) + \
           ce_loss(tgt, depth_rgb_alpha, args.n_classes, i_epoch, args.annealing_epoch)
    return loss, depth_alpha, rgb_alpha, depth_rgb_alpha, tgt


def model_eval(i_epoch, data, model, args, criterion):
    model.eval()
    with torch.no_grad():
        losses, depth_preds, rgb_preds, depthrgb_preds, tgts = [], [], [], [], []
        for batch in data:
            loss, depth_alpha, rgb_alpha, depth_rgb_alpha, tgt = model_forward(i_epoch, model, args, criterion, batch)
            losses.append(loss.item())

            depth_pred = depth_alpha.argmax(dim=1).cpu().detach().numpy()
            rgb_pred = rgb_alpha.argmax(dim=1).cpu().detach().numpy()
            depth_rgb_pred = depth_rgb_alpha.argmax(dim=1).cpu().detach().numpy()

            depth_preds.append(depth_pred)
            rgb_preds.append(rgb_pred)
            depthrgb_preds.append(depth_rgb_pred)
            tgt = tgt.cpu().detach().numpy()
            tgts.append(tgt)

    metrics = {"loss": np.mean(losses)}

    tgts = [l for sl in tgts for l in sl]
    depth_preds = [l for sl in depth_preds for l in sl]
    rgb_preds = [l for sl in rgb_preds for l in sl]
    depthrgb_preds = [l for sl in depthrgb_preds for l in sl]
    metrics["depth_acc"] = accuracy_score(tgts, depth_preds)
    metrics["rgb_acc"] = accuracy_score(tgts, rgb_preds)
    metrics["depthrgb_acc"] = accuracy_score(tgts, depthrgb_preds)
    metrics["depth_pre"] = precision_score(tgts, depth_preds,average='weighted')
    metrics["rgb_pre"] = precision_score(tgts, rgb_preds,average='weighted')
    metrics["depthrgb_pre"] = precision_score(tgts, depthrgb_preds,average='weighted')
    metrics["depth_rec"] = recall_score(tgts, depth_preds,average='weighted')
    metrics["rgb_rec"] = recall_score(tgts, rgb_preds,average='weighted')
    metrics["depthrgb_rec"] = recall_score(tgts, depthrgb_preds,average='weighted')
    metrics["depth_f1"] = f1_score(tgts, depth_preds,average='weighted')
    metrics["rgb_f1"] = f1_score(tgts, rgb_preds,average='weighted')
    metrics["depthrgb_f1"] = f1_score(tgts, depthrgb_preds,average='weighted')
    return metrics


def train(args):
    set_seed(args.seed)
    args.savedir = os.path.join(args.savedir, args.name)
    os.makedirs(args.savedir, exist_ok=True)
    train_transforms = list()
    train_transforms.append(transforms.Resize((args.LOAD_SIZE, args.LOAD_SIZE)))
    train_transforms.append(transforms.RandomCrop((args.FINE_SIZE, args.FINE_SIZE)))
    train_transforms.append(transforms.RandomHorizontalFlip())
    train_transforms.append(transforms.ToTensor())
    train_transforms.append(transforms.Normalize(mean=[0.6983, 0.3918, 0.4474], std=[0.1648, 0.1359, 0.1644]))
    val_transforms = list()
    val_transforms.append(transforms.Resize((args.FINE_SIZE, args.FINE_SIZE)))
    val_transforms.append(transforms.ToTensor())
    val_transforms.append(transforms.Normalize(mean=[0.6983, 0.3918, 0.4474], std=[0.1648, 0.1359, 0.1644]))

    train_loader = DataLoader(
        AlignedConcDataset(args, data_dir=os.path.join(args.data_path, 'train'), transform=transforms.Compose(train_transforms)),
        batch_size=args.batch_sz,
        shuffle=True,
        num_workers=args.n_workers)
    test_loader = DataLoader(
            AlignedConcDataset(args, data_dir=os.path.join(args.data_path, 'test'), transform=transforms.Compose(val_transforms)),
            batch_size=args.batch_sz,
            shuffle=False,
            num_workers=args.n_workers)
    model = ETMC(args)
    optimizer = get_optimizer(model, args)
    scheduler = get_scheduler(optimizer, args)
    logger = create_logger("%s/logfile.log" % args.savedir, args)
    model.cuda()

    torch.save(args, os.path.join(args.savedir, "args.pt"))
    start_epoch, global_step, n_no_improve, best_metric = 0, 0, 0, -np.inf

    #创建结果的csv格式输出文件
    #dataset = "nyudrgbd"
    filepath = 'results/HD_ETMC/nyud2_1_{}_{}_{}.csv'.format(args.backbone,args.Holder_parge,args.data_path.split('/')[-2])
    #以新建的方式（会对原有文件进行覆盖），创建文件
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    f = open(filepath, "w")
    f.writelines("loss,depth_acc,rgb_acc,depth_rgb_acc\n")

    if os.path.exists(os.path.join(args.savedir, "checkpoint.pt")):
        checkpoint = torch.load(os.path.join(args.savedir, "checkpoint.pt"))
        start_epoch = checkpoint["epoch"]
        n_no_improve = checkpoint["n_no_improve"]
        best_metric = checkpoint["best_metric"]
        model.load_state_dict(checkpoint["state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        scheduler.load_state_dict(checkpoint["scheduler"])

    for i_epoch in range(start_epoch, args.max_epochs):
        train_losses = []
        model.train()
        optimizer.zero_grad()
        for batch in tqdm(train_loader, total=len(train_loader)):
            loss, depth_out, rgb_out, depthrgb, tgt = model_forward(i_epoch, model, args, ce_loss, batch)
            if args.gradient_accumulation_steps > 1:
                 loss = loss / args.gradient_accumulation_steps

            train_losses.append(loss.item())
            loss.backward()
            global_step += 1
            if global_step % args.gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

        model.eval()
        metrics = model_eval(
            np.inf, test_loader, model, args, ce_loss
        )
        logger.info("Train Loss: {:.4f}".format(np.mean(train_losses)))
        # log_metrics("val", metrics, logger)
        # logger.info(
        #     "{}: Loss: {:.5f} | depth_acc: {:.5f}, rgb_acc: {:.5f}, depth rgb acc: {:.5f}".format(
        #         "val", metrics["loss"], metrics["depth_acc"], metrics["rgb_acc"], metrics["depthrgb_acc"]
        #     )
        # )
        metric_types = ["acc", "pre", "rec", "f1"]
        metric_prefixes = ["depth", "rgb", "depthrgb"]

        for metric_type in metric_types:
            metric_values = [metrics[f"{prefix}_{metric_type}"] for prefix in metric_prefixes]
            log_message = "{}: Loss: {:.5f} | {}: {:.5f}, {}: {:.5f}, {}: {:.5f}".format(
                "Test", metrics["loss"],
                f"depth_{metric_type}", metric_values[0],
                f"rgb_{metric_type}", metric_values[1],
                f"depthrgb_{metric_type}", metric_values[2]
            )
            logger.info(log_message)
        tuning_metric = metrics["depthrgb_acc"]

        #向csv格式文件写入数据
        f.writelines(f"{metrics['loss']},{metrics['depth_acc']},{metrics['rgb_acc']},{metrics['depthrgb_acc']}\n")

        scheduler.step(tuning_metric)
        is_improvement = tuning_metric > best_metric
        if is_improvement:
            best_metric = tuning_metric
            n_no_improve = 0
        else:
            n_no_improve += 1

        save_checkpoint(
            {
                "epoch": i_epoch + 1,
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "n_no_improve": n_no_improve,
                "best_metric": best_metric,
            },
            is_improvement,
            args.savedir,
        )

        if n_no_improve >= args.patience:
            logger.info("No improvement. Breaking out of loop.")
            break

    load_checkpoint(model, os.path.join(args.savedir, "model_best.pt"))
    model.eval()
    test_metrics = model_eval(
        np.inf, test_loader, model, args, ce_loss
    )
    logger.info(
        "{}: Loss: {:.5f} | depth_acc: {:.5f}, rgb_acc: {:.5f}, depth rgb acc: {:.5f}".format(
            "Test", test_metrics["loss"], test_metrics["depth_acc"], test_metrics["rgb_acc"],
            test_metrics["depthrgb_acc"]
        )
    )
    logger.info(
        "{}: Loss: {:.5f} | depth_pre: {:.5f}, rgb_pre: {:.5f}, depth rgb pre: {:.5f}".format(
            "Test", test_metrics["loss"], test_metrics["depth_pre"], test_metrics["rgb_pre"],
            test_metrics["depthrgb_pre"]
        )
    )
    logger.info(
        "{}: Loss: {:.5f} | depth_rec: {:.5f}, rgb_rec: {:.5f}, depth rgb rec: {:.5f}".format(
            "Test", test_metrics["loss"], test_metrics["depth_rec"], test_metrics["rgb_rec"],
            test_metrics["depthrgb_rec"]
        )
    )
    logger.info(
        "{}: Loss: {:.5f} | depth_f1: {:.5f}, rgb_f1: {:.5f}, depth rgb f1: {:.5f}".format(
            "Test", test_metrics["loss"], test_metrics["depth_f1"], test_metrics["rgb_f1"],
            test_metrics["depthrgb_f1"]
        )
    )
    f.writelines("loss,best_depth_acc,best_rgb_acc,best_depth_rgb_acc\n")
    f.writelines(f"{test_metrics['loss']},{test_metrics['depth_acc']},{test_metrics['rgb_acc']},{test_metrics['depthrgb_acc']}\n")
    f.close()
    log_metrics(f"Test", test_metrics, logger)


def cli_main():
    parser = argparse.ArgumentParser(description="Train Models")
    get_args(parser)
    args =parser.parse_known_args()[0]
    print(args)
    #args, remaining_args = parser.parse_known_args()
    #print(remaining_args)
    #assert remaining_args == [], remaining_args
    train(args)


if __name__ == "__main__":
    import warnings
    warnings.filterwarnings("ignore")
    cli_main()



Namespace(batch_sz=32, data_path='/content/gdrive/My Drive/ETMC/datasets/nyud2/', LOAD_SIZE=256, FINE_SIZE=224, dropout=0.1, gradient_accumulation_steps=3, hidden=[], hidden_sz=768, img_embed_pool_type='avg', img_hidden_sz=512, include_bn=True, lr=0.0005, lr_factor=0.2, lr_patience=30, max_epochs=500, n_workers=12, name='resReleasedVersion', num_image_embeds=1, patience=30, savedir='/content/gdrive/My Drive/ETMC/results/nyud2/', seed=1, n_classes=10, annealing_epoch=10, Holder_parge=1.5, backbone='Res')


INFO - 01/10/25 03:21:49 - 0:00:00 - FINE_SIZE: 224
                                     Holder_parge: 1.5
                                     LOAD_SIZE: 256
                                     annealing_epoch: 10
                                     backbone: Res
                                     batch_sz: 32
                                     data_path: /content/gdrive/My Drive/ETMC/datasets/nyud2/
                                     dropout: 0.1
                                     gradient_accumulation_steps: 3
                                     hidden: []
                                     hidden_sz: 768
                                     img_embed_pool_type: avg
                                     img_hidden_sz: 512
                                     include_bn: True
                                     lr: 0.0005
                                     lr_factor: 0.2
                                     lr_patience: 30
                                     max_epoch